In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import catboost
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool, cv
from pathlib import Path
import src.utils as u
import src.cb_utils as m
from tqdm.auto import tqdm
from collections import Counter
from sklearn.metrics import f1_score, precision_score, recall_score

pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 100)

u.random_seed(42)

data_path = Path('../data')

In [ ]:
data_path = Path('../data')

train_full, test, all_cols, cont_cols, cat_cols = u.read_data(data_path)

In [ ]:
train = u.select_by_time(train_full_high, '2016-10-01', '2018-10-01', 'datetime')
val = u.select_by_time(train_full_high, '2018-10-01', '2019-01-01', 'datetime')

train_pool = Pool(data=train[all_cols], label=train['y'], cat_features=cat_cols)
val_pool =  Pool(data=val[all_cols], label=val['y'], cat_features=cat_cols)
test_pool = Pool(data=test[all_cols], cat_features=cat_cols)

In [ ]:
params = {
    'iterations': 500,
    'learning_rate': 0.1,
    'scale_pos_weight': 5,
    'has_time': True,
    'one_hot_max_size': 1000,
    'depth': 5,
    
    'loss_function': 'Logloss',
    'task_type': 'CPU',
    'use_best_model': True,
    'eval_metric': m.FlexibleF1(0, 1, 20)
}

In [ ]:
cls = CatBoostClassifier(**params)

cls.fit(
    train_pool,
    eval_set=val_pool,
    plot=True,
    verbose=1,
    early_stopping_rounds=40,
)

# cls.save_model('../results/cb_model.pt')

In [ ]:
th = u.estimate(cls, val_pool, y_true=val_pool.get_label(), th_start=0, th_stop=1, steps=100)

In [ ]:
proba_test = cls.predict_proba(test_pool)[:, 1]

# Submit

In [ ]:
pred = proba_test > 0.285

n_pred = sum(pred)
print(n_pred)

In [ ]:
submit = pd.DataFrame(
    data={'datetime x segment_id': test['datetime x segment_id'].values,
          'prediction': pred.astype(int)}
)
submit.to_csv(f'../results/submit_cb_{n_pred}.csv', index=False)